# Generate Keywords Ground Truth
This notebook generates the ground truth dataset for search keywords, given user input.

In [1]:
from multiprocessing import *

import const
import ctypes
import numpy
import pandas as pd
import traceback
import yake

In [2]:
temp_squad_ds = pd.read_csv(const.DATASETS_FOLDER + "squad-train-v2.0.csv")

In [3]:
k = Value(ctypes.py_object)
k.value = temp_squad_ds

In [4]:
print(k.value)

        Unnamed: 0     source    topic  \
0                0  Squad 2.0  Beyoncé   
1                1  Squad 2.0  Beyoncé   
2                2  Squad 2.0  Beyoncé   
3                3  Squad 2.0  Beyoncé   
4                4  Squad 2.0  Beyoncé   
...            ...        ...      ...   
130314      130314  Squad 2.0   Matter   
130315      130315  Squad 2.0   Matter   
130316      130316  Squad 2.0   Matter   
130317      130317  Squad 2.0   Matter   
130318      130318  Squad 2.0   Matter   

                                                paragraph  \
0       Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
1       Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
2       Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
3       Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
4       Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...   
...                                                   ...   
130314  The term "matter" is used throughout physics i...   
130315 

In [5]:
def generate_keywords(squad_ds_ns, start_row, end_row, batch_size = 64):
    #print("start: " + str(start_row) + ", end: " + str(end_row))
    temp_squad_ds = squad_ds_ns.df
    row_len = end_row - start_row
    if row_len <= batch_size:
        kw_extr = yake.KeywordExtractor(n=16)
        for row in range(start_row, end_row):
            keywords = kw_extr.extract_keywords(temp_squad_ds.loc[row, "question"])
            if keywords == []:
                    print("No keywords extracted at row " + str(row) + " (question: " + temp_squad_ds.loc[row, "question"] + ")")
            else:
                temp_squad_ds.loc[row, "keyword"] = keywords[0][0]
            alt_keywords = []
            for item in keywords:
                alt_keywords.append(item[0])
            temp_squad_ds.loc[row, "possible_keywords"] = str(alt_keywords)
        squad_ds_ns.df = temp_squad_ds
    else:
        row_range = end_row - start_row
        left_start = start_row
        left_end = int((row_range)/2) + left_start
        right_start = start_row + int((row_range)/2)
        right_end = end_row

        left_mgr = Manager()
        right_mgr = Manager()
        left_squad_ns = left_mgr.Namespace()
        right_squad_ns = right_mgr.Namespace()

        left_squad_ns.df = squad_ds_ns.df[0:int(row_range/2)].copy()
        right_squad_ns.df = squad_ds_ns.df[int(row_range/2):row_range].copy()
        p_left = Process(target=generate_keywords, args=(left_squad_ns, left_start, left_end, batch_size))
        p_right = Process(target=generate_keywords, args=(right_squad_ns, right_start, right_end, batch_size))
        p_left.start()
        p_right.start()
        p_left.join()
        p_right.join()
        squad_ds_ns.df = pd.concat([left_squad_ns.df, right_squad_ns.df])

In [6]:
temp_squad_ds.head()

,Unnamed: 0,source,topic,paragraph,question,question_id,is_impossible,answers,expanded_answers
0,0,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,False,in the late 1990s,in the late 1990s as lead singer of R&B girl-g...
1,1,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,False,singing and dancing,in the late 1990s as lead singer of R&B girl-g...
2,2,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,False,2003,in the late 1990s as lead singer of R&B girl-g...
3,3,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,False,"Houston, Texas",in the late 1990s as lead singer of R&B girl-g...
4,4,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,False,late 1990s,in the late 1990s as lead singer of R&B girl-g...


In [7]:
end = len(temp_squad_ds)
mgr = Manager()
squad_ns = mgr.Namespace()
squad_ns.df = temp_squad_ds
generate_keywords(squad_ns, 0, len(squad_ns.df), batch_size=1024)
squad_ds = squad_ns.df

No keywords extracted at row 16302 (question: What is the BeiDou-1?)
No keywords extracted at row 16363 (question: What was BeiDou-3 formerly known as?)
No keywords extracted at row 28529 (question: Where are most of them from?)
No keywords extracted at row 73339 (question: Why was that? )
No keywords extracted at row 26517 (question: What came out in 1935?)
No keywords extracted at row 73358 (question: Did that used to have a different name?)
No keywords extracted at row 12289 (question: What is Sarva-darśana-samgraha?)
No keywords extracted at row 65215 (question: What novel came out in 2008?)
No keywords extracted at row 8208 (question: What came into use in the 1800's?)
No keywords extracted at row 16484 (question: What is the Expedition-1?)
No keywords extracted at row 79512 (question: Where does its name come from?)
No keywords extracted at row 86636 (question: Both p2 and p3 are what?)
No keywords extracted at row 37828 (question: What was it used for?)
No keywords extracted at 

In [8]:
squad_ds.head()

,Unnamed: 0,source,topic,paragraph,question,question_id,is_impossible,answers,expanded_answers,keyword,possible_keywords
0,0,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce start becoming popular?,56be85543aeaaa14008c9063,False,in the late 1990s,in the late 1990s as lead singer of R&B girl-g...,Beyonce start becoming popular,"['Beyonce start becoming popular', 'Beyonce st..."
1,1,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,What areas did Beyonce compete in when she was...,56be85543aeaaa14008c9065,False,singing and dancing,in the late 1990s as lead singer of R&B girl-g...,areas did Beyonce compete in when she was growing,['areas did Beyonce compete in when she was gr...
2,2,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,When did Beyonce leave Destiny's Child and bec...,56be85543aeaaa14008c9066,False,2003,in the late 1990s as lead singer of R&B girl-g...,Beyonce leave Destiny Child and become a solo ...,['Beyonce leave Destiny Child and become a sol...
3,3,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In what city and state did Beyonce grow up?,56bf6b0f3aeaaa14008c9601,False,"Houston, Texas",in the late 1990s as lead singer of R&B girl-g...,city and state did Beyonce grow,"['city and state did Beyonce grow', 'city and ..."
4,4,Squad 2.0,Beyoncé,Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ b...,In which decade did Beyonce become famous?,56bf6b0f3aeaaa14008c9602,False,late 1990s,in the late 1990s as lead singer of R&B girl-g...,decade did Beyonce become famous,"['decade did Beyonce become famous', 'Beyonce ..."


In [9]:
squad_ds.tail()

,Unnamed: 0,source,topic,paragraph,question,question_id,is_impossible,answers,expanded_answers,keyword,possible_keywords
130314,130314,Squad 2.0,Matter,"The term ""matter"" is used throughout physics i...",Physics has broadly agreed on the definition o...,5a7e070b70df9f001a875439,True,matter,"ter and antimatter, normal matter has been ref...",Physics has broadly agreed on the definition,['Physics has broadly agreed on the definition...
130315,130315,Squad 2.0,Matter,"The term ""matter"" is used throughout physics i...",Who coined the term partonic matter?,5a7e070b70df9f001a87543a,True,Alfvén,"ter and antimatter, normal matter has been ref...",coined the term partonic matter,"['coined the term partonic matter', 'term part..."
130316,130316,Squad 2.0,Matter,"The term ""matter"" is used throughout physics i...",What is another name for anti-matter?,5a7e070b70df9f001a87543b,True,Gk. common matter,"ter and antimatter, normal matter has been ref...",anti-matter,['anti-matter']
130317,130317,Squad 2.0,Matter,"The term ""matter"" is used throughout physics i...",Matter usually does not need to be used in con...,5a7e070b70df9f001a87543c,True,a specifying modifier,"ter and antimatter, normal matter has been ref...",Matter usually does not need to be used in con...,['Matter usually does not need to be used in c...
130318,130318,Squad 2.0,Matter,"The term ""matter"" is used throughout physics i...",What field of study has a variety of unusual c...,5a7e070b70df9f001a87543d,True,physics,"ter and antimatter, normal matter has been ref...",field of study has a variety of unusual contexts,['field of study has a variety of unusual cont...


In [10]:
squad_ds['keyword']

0                            Beyonce start becoming popular
1         areas did Beyonce compete in when she was growing
2         Beyonce leave Destiny Child and become a solo ...
3                           city and state did Beyonce grow
4                          decade did Beyonce become famous
                                ...                        
130314         Physics has broadly agreed on the definition
130315                      coined the term partonic matter
130316                                          anti-matter
130317    Matter usually does not need to be used in con...
130318     field of study has a variety of unusual contexts
Name: keyword, Length: 130319, dtype: object

In [11]:
squad_ds.to_csv(const.DATASETS_FOLDER + "squad_ds_keyword_ground_truth.csv")